In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.rnn import BasicRNNCell
from tensorflow.examples.tutorials.mnist import input_data


import numpy as np
import os
from datetime import datetime

In [2]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [3]:
learning_rate = 0.001

In [4]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = fully_connected(states, n_outputs, activation_fn=None)

with tf.name_scope('loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='x_entropy')
    loss = tf.reduce_mean(x_entropy, name ='loss')

learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


In [5]:

now = datetime.utcnow().strftime("%Y-%m-%d-%H:%M:%S")
root_logdir = "tf_logs"
logdir = os.path.join(os.getcwd(),"{}/run-{}".format(root_logdir, now))

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [6]:
n_epochs = 10
batch_size = 50
mnist = input_data.read_data_sets("tmp/data")

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
X_test = mnist.test.images.reshape(-1, n_steps, n_inputs)
y_test = mnist.test.labels

In [8]:
accuracy_summary = tf.summary.scalar('Accuracy', accuracy)

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

sess = tf.InteractiveSession()
init.run()

for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        X_batch = X_batch.reshape(-1, n_steps, n_inputs)
        sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:X_test, y:mnist.test.labels})
    file_writer.add_summary(accuracy_summary.eval(feed_dict={X:X_test, y:mnist.test.labels}),epoch)
    
    print(epoch, "Train accuracy ", acc_train, "Test accuracy ", acc_test)

save_path = saver.save(sess, os.path.join(os.getcwd(),"Models/MNIST_RNN_Model.ckpt"))

0 Train accuracy  0.86 Test accuracy  0.9213
1 Train accuracy  0.94 Test accuracy  0.8927
2 Train accuracy  0.96 Test accuracy  0.9457
3 Train accuracy  1.0 Test accuracy  0.9621
4 Train accuracy  0.96 Test accuracy  0.9642
5 Train accuracy  1.0 Test accuracy  0.9638
6 Train accuracy  1.0 Test accuracy  0.9623
7 Train accuracy  0.98 Test accuracy  0.9662
8 Train accuracy  1.0 Test accuracy  0.9658
9 Train accuracy  0.98 Test accuracy  0.9737


In [10]:
sess.close()
file_writer.close()